In [1]:
import datetime
import configparser

from kiteconnect import KiteTicker
from kiteconnect import KiteConnect

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import os.path
from os import system

import json
import pyotp
import requests
import numpy as np
import pandas as pd
from time import sleep
import urllib.parse as urlparse

import telegram

In [3]:
def login(user_id, method='standard'):
    """
    login method can be jugaad or standard
    """
    
    #give path to credentials file here
    credentials_file = pd.read_csv('credentials.csv')

    method = credentials_file[credentials_file['user_id'] == user_id]['method'].iloc[0]

    if method == 'standard':


        api_key = credentials_file[credentials_file['user_id'] == user_id]['api_key'].iloc[0]
        api_secret = credentials_file[credentials_file['user_id'] == user_id]['api_secret'].iloc[0]
        account_username = credentials_file[credentials_file['user_id'] == user_id]['user_id'].iloc[0]
        account_password = credentials_file[credentials_file['user_id'] == user_id]['password'].iloc[0]
        totp = credentials_file[credentials_file['user_id'] == user_id]['totp'].iloc[0]
        auth_key = pyotp.TOTP(totp)

        kite = KiteConnect(api_key=api_key)

        #check whether today's date is the same as the date saved in access token date in csv
        #checking for 2 date formats as sometimes after opening csv, excel changes format
        if ((credentials_file[credentials_file['user_id'] == user_id]['access_token_date'].iloc[0] == datetime.datetime.today().date().strftime('%d-%m-%Y')) | (credentials_file[credentials_file['user_id'] == user_id]['access_token_date'].iloc[0] == str(datetime.datetime.today().date()))):
            access_token = credentials_file[credentials_file['user_id'] == user_id]['access_token'].iloc[0]
            print(f'Access Token Generated Earlier Today for {user_id}')


        else:
            options = Options()
            options.add_argument('--headless')
            options.add_argument('--disable-gpu')
            driver = webdriver.Chrome(executable_path='chromedriver.exe',options=options)
            driver.get(kite.login_url())
            sleep(2)
            form = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="login-form"]')))
            driver.find_element("xpath","//input[@type='text']").send_keys(account_username)
            driver.find_element("xpath","//input[@type='password']").send_keys(account_password)

            driver.find_element("xpath","//span[@class='su-checkbox-box']").click()
            driver.find_element("xpath","//button[@type='submit']").click()
            sleep(2)
            form = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="login-form"]//form')))
            driver.find_element("xpath","//input[@type='text']").send_keys(auth_key.now())
            driver.find_element("xpath","//button[@type='submit']").click()
            sleep(2)

            current_url = driver.current_url
            driver.close()

            parsed = urlparse.urlparse(current_url)
            request_token = urlparse.parse_qs(parsed.query)['request_token'][0]
            access_token = kite.generate_session(request_token=request_token,api_secret=api_secret)['access_token']

            #Writing access token, date and object to credentials file
            credentials_file.loc[credentials_file['user_id'] == user_id,'access_token_date'] = str(datetime.datetime.today().date())
            credentials_file.loc[credentials_file['user_id'] == user_id,'access_token'] = access_token
            credentials_file.loc[credentials_file['user_id'] == user_id,'object'] = kite

            #save the config file
            try:
                credentials_file.to_csv("credentials.csv",index=False)
                print("Saved credentials csv for",user_id)
            except:
                print(f"Error while saving {user_id} access token to csv. Maybe the credentials csv file is open.")
                

                
        login_success_counter = 0 #to track whether account got logged in or not
        
        try:
            kite.set_access_token(access_token)
            kite_login_message = str(f"Login success for ID: {kite.profile()['user_id']}")
            print(kite_login_message)
            login_success_counter = "Yes"
        except:
            kite_login_message = str(f"ERROR logging into ID: {kite.profile()['user_id']}")
            print(kite_login_message)
            login_success_counter = "No"
            
    elif method == 'jugaad':
        pass
    
    return kite,kite_login_message,login_success_counter

In [4]:
credentials = pd.read_csv("credentials.csv")

In [5]:
credentials

,user_id,password,method,api_key,api_secret,totp,access_token_date,access_token,object
0,VR2386,Maini!@345,standard,nnd1q4dbjvhyxqc3,vrrqogov68e24waj9w6hsby0f8ivj8mc,VLPPKCKD2KW44KJXLEM5KMREVWJTQX6S,2023-03-16,nDeOTIheCQhI71pnk7fqMBtC51cuOhk3,<kiteconnect.connect.KiteConnect object at 0x0...
1,AJ1440,Target1cr,standard,9hduta0up86dg9yk,x3o8wdc59a0ccmmy3lvh496bhhr9avz3,UOYQB432M6JBJDFB66TO2C7TWUFYLHS4,2023-03-16,PdTnEeEZWutE6v9BkMzt454uQEaCK1yg,<kiteconnect.connect.KiteConnect object at 0x0...


In [6]:
accounts = credentials['user_id'].to_list()

In [7]:
for user_id in accounts:
    if user_id is np.nan:
        continue
    try:
        account_login = login(user_id)

    except:
        print("Error logging into: ",user_id)


C:\Users\ARJUNT~1\AppData\Local\Temp/ipykernel_11808/1869299642.py:34: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='chromedriver.exe',options=options)


Saved credentials csv for VR2386
Login success for ID: VR2386


C:\Users\ARJUNT~1\AppData\Local\Temp/ipykernel_11808/1869299642.py:34: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='chromedriver.exe',options=options)


Saved credentials csv for AJ1440
Login success for ID: AJ1440


In [8]:
credentials = pd.read_csv("credentials.csv")

In [9]:
credentials

,user_id,password,method,api_key,api_secret,totp,access_token_date,access_token,object
0,VR2386,Maini!@345,standard,nnd1q4dbjvhyxqc3,vrrqogov68e24waj9w6hsby0f8ivj8mc,VLPPKCKD2KW44KJXLEM5KMREVWJTQX6S,2023-03-20,ASjSow6by68zbcMqyl7PrODpJbykdIsI,<kiteconnect.connect.KiteConnect object at 0x0...
1,AJ1440,Target1cr,standard,9hduta0up86dg9yk,x3o8wdc59a0ccmmy3lvh496bhhr9avz3,UOYQB432M6JBJDFB66TO2C7TWUFYLHS4,2023-03-20,GAV2unIHqeDgsnlY7L2ZL80YoC1mlL7l,<kiteconnect.connect.KiteConnect object at 0x0...


In [10]:
def login_all():
    #login process has already been done when initialising Raptor, but this process check login status once again
    #iterate over the accounts in the accounts dictionary and login each one of them, in case not already done
    #in case unable to login, print an error message and also send to the group
    
    for user_id in credentials['user_id'].to_list():
        try:
            credentials.loc[credentials['user_id'] == user_id,'object'] = login(user_id,credentials[credentials['user_id'] == user_id]['method'].iloc[0])[0]
            print('success')
        except:
            print("Could not login to",user_id)

login_all()

Access Token Generated Earlier Today for VR2386
Login success for ID: VR2386
success
Access Token Generated Earlier Today for AJ1440
Login success for ID: AJ1440
success


In [12]:
credentials['object'].iloc[0]

In [83]:
kite.margins()

{'equity': {'enabled': True,
  'net': 7507612.191,
  'available': {'adhoc_margin': 0,
   'cash': 3024457.6,
   'opening_balance': 3024457.6,
   'live_balance': 2872378.85,
   'collateral': 5436342,
   'intraday_payin': 0},
  'utilised': {'debits': 953187.409,
   'exposure': 122848.739,
   'm2m_realised': 0,
   'm2m_unrealised': 0,
   'option_premium': -36299.375,
   'payout': 0,
   'span': 678259.92,
   'holding_sales': 0,
   'turnover': 0,
   'liquid_collateral': 959846.8,
   'stock_collateral': 4476495.2,
   'equity': 0,
   'delivery': 0}},
 'commodity': {'enabled': False,
  'net': 0,
  'available': {'adhoc_margin': 0,
   'cash': 0,
   'opening_balance': 0,
   'live_balance': 0,
   'collateral': 0,
   'intraday_payin': 0},
  'utilised': {'debits': 0,
   'exposure': 0,
   'm2m_realised': 0,
   'm2m_unrealised': 0,
   'option_premium': 0,
   'payout': 0,
   'span': 0,
   'holding_sales': 0,
   'turnover': 0,
   'liquid_collateral': 0,
   'stock_collateral': 0,
   'equity': 0,
   'deliv